In [1]:
import urllib.request
import urllib.parse
import xml.etree.ElementTree as ET

def search_arxiv(query_text, max_results=10):
    """
    ArXiv API를 사용하여 논문을 검색하고 결과를 반환합니다.

    Args:
        query_text (str): 검색할 키워드 (예: "MLOps AND LLM", "MLOps OR machine learning operations").
                          AND, OR, NOT 연산자는 대문자로 사용하며, 공백으로 구분합니다.
    Returns:
        str: ArXiv API로부터 받은 XML 데이터.
    """
    # 쿼리 텍스트를 URL에 안전한 형태로 인코딩 (공백을 '+'로 변환)
    # ArXiv API는 검색어 내의 공백을 AND 연산자로 해석합니다.
    # 명시적으로 AND/OR/NOT을 사용할 때도 이 인코딩 방식이 맞습니다.
    encoded_query = urllib.parse.quote_plus(query_text)
    
    base_url = 'http://export.arxiv.org/api/query?'
    query = f'search_query={encoded_query}&start=0&max_results={max_results}'
    
    url = base_url + query
    
    print(f"DEBUG: Requesting URL: {url}") # 디버깅을 위해 요청 URL 출력
    
    try:
        with urllib.request.urlopen(url) as response:
            data = response.read().decode('utf-8')
            return data
    except Exception as e:
        print(f"API 요청 중 오류 발생: {e}")
        return None

# parse_arxiv_results 함수는 이전과 동일합니다.
def parse_arxiv_results(xml_data):
    if not xml_data:
        return 0, []

    root = ET.fromstring(xml_data)

    namespaces = {
        'atom': 'http://www.w3.org/2005/Atom',
        'opensearch': 'http://a9.com/-/spec/opensearch/1.1/'
    }

    total_results_element = root.find('opensearch:totalResults', namespaces)
    total_results = int(total_results_element.text) if total_results_element is not None else 0

    articles = []
    for entry in root.findall('atom:entry', namespaces):
        title_element = entry.find('atom:title', namespaces)
        summary_element = entry.find('atom:summary', namespaces)
        published_element = entry.find('atom:published', namespaces)
        
        title = title_element.text if title_element is not None else "제목 없음"
        summary = summary_element.text.strip() if summary_element is not None else "요약 없음"
        published = published_element.text if published_element is not None else "날짜 없음"
        
        authors = [author.find('atom:name', namespaces).text for author in entry.findall('atom:author', namespaces) if author.find('atom:name', namespaces) is not None]
        
        articles.append({
            'title': title,
            'summary': summary,
            'published': published,
            'authors': authors
        })
    return total_results, articles

# --- 사용 예시 ---
if __name__ == "__main__":
    print("MLOps와 LLM 관련 논문 검색 도우미입니다.")
    print("검색어는 **AND, OR, NOT** 연산자를 대문자로 사용하며, **공백**으로 구분합니다.")
    print("예시: 'all:MLOps AND LLM'")
    print("예시: 'all:MLOps AND LLM AND (deployment OR monitoring)'")
    
    user_raw_query = input("검색 쿼리를 입력하세요: ")
    
    # 쿼리 앞부분에 'all:'이 없으면 기본적으로 all:로 검색됩니다.
    # 명시적으로 all:을 붙여주는 것이 좋습니다.
    if not user_raw_query.strip().startswith("all:"):
        user_raw_query = "all:" + user_raw_query

    num_results_to_fetch = int(input("가져올 최대 결과 개수를 입력하세요 (예: 5): "))

    print(f"'{user_raw_query}'(으)로 ArXiv 검색 중...")
    xml_output = search_arxiv(user_raw_query, max_results=num_results_to_fetch)

    if xml_output:
        total_count, articles_list = parse_arxiv_results(xml_output)
        print(f"\n--- 검색 결과 (총 {total_count}개 중 {len(articles_list)}개 표시) ---")
        if articles_list:
            for i, article in enumerate(articles_list):
                print(f"\n{i+1}. 제목: {article['title']}")
                print(f"   저자: {', '.join(article['authors'])}")
                print(f"   발행일: {article['published']}")
                print(f"   요약: {article['summary'][:250]}...") # 요약이 길 경우 일부만 표시
        else:
            print("검색 결과가 없습니다.")
    else:
        print("검색 결과를 가져오는 데 실패했습니다.")

MLOps와 LLM 관련 논문 검색 도우미입니다.
검색어는 **AND, OR, NOT** 연산자를 대문자로 사용하며, **공백**으로 구분합니다.
예시: 'all:MLOps AND LLM'
예시: 'all:MLOps AND LLM AND (deployment OR monitoring)'
'all:MLOps AND LLM'(으)로 ArXiv 검색 중...
DEBUG: Requesting URL: http://export.arxiv.org/api/query?search_query=all%3AMLOps+AND+LLM&start=0&max_results=5

--- 검색 결과 (총 8개 중 5개 표시) ---

1. 제목: Automating Code Adaptation for MLOps -- A Benchmarking Study on LLMs
   저자: Harsh Patel, Buvaneswari A. Ramanan, Manzoor A. Khan, Thomas Williams, Brian Friedman, Lawrence Drabeck
   발행일: 2024-05-10T22:18:43Z
   요약: This paper explores the possibilities of the current generation of Large
Language Models for incorporating Machine Learning Operations (MLOps)
functionalities into ML training code bases. We evaluate the performance of
OpenAI (gpt-3.5-turbo) and Wiza...

2. 제목: DNN-Powered MLOps Pipeline Optimization for Large Language Models: A
  Framework for Automated Deployment and Resource Management
   저자: Mahesh Vaijainthymala Krishnamoorthy